# 1. Problem Statement
---

# 2. Functions Preparation
---

## 2.1 Libraries Importation

In [5]:
# Import libraries

# General libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Model


## 2.2 In-notebook Functions Creation

# 3. Data Preparation
---

## 3.1 Data Consolidation

In [10]:
df = pd.DataFrame()

# Set relative path for raw data folder
relative_path = ".\selenium\scrape_data"

# Get file names in folder as a list
file_names = os.listdir(relative_path)

# Iterate raw csv files in folder
for file_name in file_names:
    try: 
        temp_df = pd.read_csv(f"{relative_path}\\{file_name}")
        df = pd.concat([df, temp_df])
    except:
        # Print error file name
        print(file_name)

## 3.2 Data Cleansing

### 3.2.1 Duplication Removal

In [21]:
# Check data shape
# There are 1020 rows and 21 features from raw datas
df.shape

(1020, 21)

In [23]:
# Check duplicate data
# Dataset contains 8 duplicate rows, we can drop it.
df.duplicated().sum()

8

In [24]:
# Drop duplicate rows and check result
df.drop_duplicates(inplace=True)
df.shape

(1012, 21)

### 3.2.2 Features Preparation
From scraping process, some features were collected as list of features due to uncertainty of the features occurrence that make .csv unable to store it in one column. 
<br>In-order-that, a list of features were joined by `|` to be a single string.

For example, column `internal_feature_keys` and `internal_feature_values` contain information of `number of bedrooms`, `number of bathrooms`, and `internal area` together as
<br>`bedrooms|bathrooms|internal area`.

**Hence, this section is to split the joined features to be ready for analyzing.**

In [30]:
df.head(2)

,dev,address,district,sale_price,rent_price,internal_feature_keys,internal_feature_values,external_feature_keys,exteranal_feature_values,description,...,amenity_keys,amenity_values,neighbor_cats,neighbor_names,neighbor_distances,asking_price,asking_price_change_quater,asking_price_change_year,gross_rental_yield,rental_price_change_year
0,H Sukhumvit 43,"Soi Phrom Mit, Khlong Toei Nuea, Vadhana, Bang...",Watthana,12000000.0,57000.0,Bedrooms|Bathrooms|Internal area,2|2|59,Floors|Towers|Project Area|Year built,"32|1|3,080 m2|2014",PropertyScout ID 954486\nCheck all listing det...,...,Elevator|Parking|24-hours Security|CCTV|Swimmi...,True|True|True|True|True|True|True|False|False...,marker marker-bts|marker marker-bts|marker mar...,Phrom Phong|Thong Lo|Sukhumvit|Asok 4 Toll Pla...,830 m / 10 minutes by foot|1.5 km / 8 minutes ...,"195,125",0.0,0.74,4.67,-6.06
1,A Space I.D. Asoke - Ratchada,"Thanon Din Daeng, Din Daeng, Bangkok 10400",Ratchathewi,4639200.0,15000.0,Bedrooms|Bathrooms|Internal area,1|1|33,Floors|Towers|Year built,30|1|2016,PropertyScout ID 954280\nCheck all listing det...,...,Elevator|Parking|24-hours Security|CCTV|Swimmi...,True|True|True|True|True|True|True|True|False|...,marker marker-mrt|marker marker-mrt|marker mar...,Phraram Kao 9|Phra Ram 9|Asok 2 Toll Plaza|Aso...,710 m / 9 minutes by foot|740 m / 9 minutes by...,"126,979",0.0,11.06,4.16,4.51


In [62]:
# def split funciton
def split_features(string):
    try:
        return string.split("|")
    except:
        return None

In [69]:
# List of joined feature column names
joined_features = ['internal_feature_keys', 'internal_feature_values', 'external_feature_keys', 'exteranal_feature_values',
                  'amenity_keys', 'amenity_values', 'neighbor_cats', 'neighbor_names', 'neighbor_distances']

# Iterate column names to create list
for feature in joined_features:
    df[feature] = df[feature].apply(split_features)

In [71]:
# Check result
df.head(2)

,dev,address,district,sale_price,rent_price,internal_feature_keys,internal_feature_values,external_feature_keys,exteranal_feature_values,description,...,amenity_keys,amenity_values,neighbor_cats,neighbor_names,neighbor_distances,asking_price,asking_price_change_quater,asking_price_change_year,gross_rental_yield,rental_price_change_year
0,H Sukhumvit 43,"Soi Phrom Mit, Khlong Toei Nuea, Vadhana, Bang...",Watthana,12000000.0,57000.0,"[Bedrooms, Bathrooms, Internal area]","[2, 2, 59]","[Floors, Towers, Project Area, Year built]","[32, 1, 3,080 m2, 2014]",PropertyScout ID 954486\nCheck all listing det...,...,"[Elevator, Parking, 24-hours Security, CCTV, S...","[True, True, True, True, True, True, True, Fal...","[marker marker-bts, marker marker-bts, marker ...","[Phrom Phong, Thong Lo, Sukhumvit, Asok 4 Toll...","[830 m / 10 minutes by foot, 1.5 km / 8 minute...","195,125",0.0,0.74,4.67,-6.06
1,A Space I.D. Asoke - Ratchada,"Thanon Din Daeng, Din Daeng, Bangkok 10400",Ratchathewi,4639200.0,15000.0,"[Bedrooms, Bathrooms, Internal area]","[1, 1, 33]","[Floors, Towers, Year built]","[30, 1, 2016]",PropertyScout ID 954280\nCheck all listing det...,...,"[Elevator, Parking, 24-hours Security, CCTV, S...","[True, True, True, True, True, True, True, Tru...","[marker marker-mrt, marker marker-mrt, marker ...","[Phraram Kao 9, Phra Ram 9, Asok 2 Toll Plaza,...","[710 m / 9 minutes by foot, 740 m / 9 minutes ...","126,979",0.0,11.06,4.16,4.51


In [75]:
# Spli lists to columns - internal_features
df['internal_feature_keys'].iloc[0]

['Bedrooms', 'Bathrooms', 'Internal area']

In [79]:
df['internal_feature_keys'].iloc[0].index('Bedrooms')

0

# 4. EDA
---

# 5. Modeling and Evaluation
---

# 6. Additional Section
---

# 7. Summarization
---